In [1]:
'''
Version 8 has the following features,
1) create button to enable/disable only when the PyPack recommendation tool has run and displayed the packages
2)The version 2 Enable/Disable the Run Pypack Recommendation tool button only when the file has loaded the user selection data from an excel
3)The version 3 showing child data for packs recommended
4)The version 4 has created modules called PyPackModule.py for all the functions and store in py file just like a configurator
5)The version 5 show all the pack channels in a different color which were part of user selection

6)The version 6 is trying to fix the problem of showing packs having same savings cost ending up showing which increases the pack cost and shows
overall savings in negative because of this

7) The version 7 will load the channels data in a new window to give an option of creating their own pack. All channels details
to be fetched from a dataframe stored in a new Module call PyPackMasterData.py
7.1) The version 7 will also try to house all the pack details in a data frame and test its performance
7.2) The version 7 will alos provide an option to pick the packs from an available service provider 

8)Create the own pack in a new window.
8.1) Checking for duplicate channels or invalid channels while adding to selected pack
8.2) Adding double click option to remove channels from the selected pack.
8.3) Run PyPack Recommendation tool button added to see the optimized selection

9)Creating a parent child class to make 
9.1) Creating a procedure to access the parent objects from child and vice verca

10)Creating an option for seeing the pack details for each service provider
10.1) creating an button to allow users to see all the pack details for that service provider
10.1) creating a new window to add a frame and show the pack details for the selected SP from drop down

11)Adding search capability to show pack window 
11.1) creating a search button for finding the packs which have a particular channel
11.1) creating two text controls to enter option to check for 2 channels in a pack

'''

from tkinter import *
from PIL import ImageTk,Image
import sqlite3
import queue
import threading
import time
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF
import math as mathObj

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

import shutil
import os as osObj
import Configurator
import PyPackModuleV3
import PyPackMasterData


class MainApplication(tk.Frame):
    
    def __init__(self,  master):
        super(MainApplication, self).\
            __init__(master)
        self.grid()
        self.root = master
        self.initialize_user_interface()
        return
    
    def initialize_user_interface(self):
        #************************************Initialise window and set size, title and icon begin code************************
        # Configure the root object for the Application
        self.root.title("PyPackSelector")
        self.root.iconbitmap('signal_icon.ico')
        #self.root.grid_rowconfigure(0, weight=1)
        #self.root.grid_columnconfigure(0, weight=1)
        self.root.config(background="green")
        #set window size
        #PyPackModule.SetWindowSize(self.root,"1150x500")
        self.root.geometry("1150x500")
        #dont allow resizing of the window
        self.root.resizable(False,False)
        #************************************Adding Declare & Initialise Variable begin code************************
        #Add all the Service Provider names
        #dropPackOptions = ["TataSky","Airtel","Hathway Jalahalli", "TNMultiDigiCon","KLYESTVPKDSD"]
        dropPackOptions = ["Hathway Jalahalli","TataSky"]
        varintcounter  = tk.IntVar()
        vartextmessage = tk.StringVar()
        varprogressmsg = tk.StringVar()
        varstroptmenu  = tk.StringVar()
        varblnfromfile = tk.StringVar()
        varstrfilename = tk.StringVar()
        
        #Set Serviceprovider pack option default as TataSky
        varstroptmenu.set(dropPackOptions[0])
        #Get the Tatasky HD Pack list
        PackageDataPath = Configurator.TataSkyHDPackPath 
        #set the default value for the message
        vartextmessage.set("status on file selection....")
        
        #set the from file flag as false
        varblnfromfile.set("No")
        
        #set the filename as blank
        varstrfilename.set("")
        
        #get all channels data
        dftemp = PyPackMasterData.GetAllChannelsData(pd)
        #df_mantemp = PyPackMasterData.GetAllChannelsData(pd,Configurator.MonthlyNetworkFee,MandatoryOnly='Yes')
        # Extract column names into a list
        #df_colnames = [x for x in dftemp.columns]
        df_colnames = ['Channel_Type','Channel_Name','Pack_Cost_Per_Month',
                       'Channel_Genre','Channel_Language','Choose_Yes_No']
        treeview_df = pd.DataFrame(columns=df_colnames)
        #************************************Adding Declare & Initialise Variable end code************************
        
        #************************************Adding all the controls begin code***********************************
        self.MyQuitbutton1         = tk.Button(self.root, text = "Quit", command=self.root.destroy)
                                                 #command=lambda:PyPackModule.CloseWindow(self.root,self.root,False))
        self.MyCreateReportbutton1 = tk.Button(self.root, text = "Create Report",state=tk.DISABLED,
                                               command=lambda:PyPackModuleV3.CreateReport(self.treeview2,
                                                                                          self.treeview1,
                                                                                          pd,
                                                                                          self.root,varprogressmsg,
                                                                                          varstrfilename.get()))
        self.MyFilebutton6 = tk.Button(self.root, text = "Open Your Choice data",
                                       command=lambda:PyPackModuleV3.OpenFileDialog(self.MyDataResultFrame1,
                                                                                    self.treeview2,self.treescroll2,
                                                                                    self.treeview1,self.treescroll1,
                                                                                    self.MyCreateReportbutton1,
                                                                                    osObj,self.root,mathObj,Configurator,
                                                                                    vartextmessage,varprogressmsg,
                                                                                    varblnfromfile,varstrfilename,
                                                                                    filedialog, pd,
                                                                                    self.Run_Button,
                                                                                    self.MyCreateReportbutton1))
        #MySLlabel1    = Label(root, text="",borderwidth=0,relief="solid",height=1, width=2,padx=5,pady=5)
        self.MyStatusLabel      = tk.Label(self.root, textvariable=vartextmessage, bd=1, relief='sunken',
                                           anchor='w',bg='Blue',fg='White')
        self.MyCreatePackButton = tk.Button(self.root, text = "Create Your Data",command=self.LoadNewWindow)
        self.MyShowPackButton   = tk.Button(self.root, text = "Show Packs",command=self.LoadPackWindow)
        self.MyDropDownPack     = tk.OptionMenu(self.root,varstroptmenu,*dropPackOptions,
                                                command=lambda _:PyPackModuleV3.SetPackOption(varstroptmenu,
                                                                                            self.Run_Button))

        #Create PyPackRecommendations Frame and add treeview1
        self.PyPackRecomResultFrame1 = tk.LabelFrame(self.root,text="Recommendation Result",padx=5,pady=5)
        #treeview1 = ttk.Treeview(PyPackRecomResultFrame1,columns=(1,2,3,4,5),show="headings",height=18,selectmode ='browse')
        self.treeview1 = ttk.Treeview(self.PyPackRecomResultFrame1,columns=("one", "two", "three","four","five"))

        #Creare Data Selection Frame to display choice and add treeview2
        self.MyDataResultFrame1      = tk.LabelFrame(self.root,text="Show Data From file",padx=5,pady=5)
        #treeview2 = ttk.Treeview(MyDataResultFrame1,columns=(1,2,3,4,5),show="headings",height=16,selectmode ='browse')
        self.treeview2 = ttk.Treeview(self.MyDataResultFrame1,columns=("one", "two", "three","four","five"))

        #adding vertical scrollbar
        self.treescroll1 = ttk.Scrollbar(self.PyPackRecomResultFrame1,orient="vertical",command=self.treeview1.yview)
        self.treescroll1.pack(side=RIGHT, fill="y") 
        self.treescroll2 = ttk.Scrollbar(self.MyDataResultFrame1,orient="vertical",command=self.treeview2.yview)
        self.treescroll2.pack(side=RIGHT, fill="y")

        #Configure the treeview to scrollback
        self.treeview1.pack() 
        self.treeview1.configure(yscrollcommand=self.treescroll1.set)
        self.treeview2.pack()
        self.treeview2.configure(yscrollcommand=self.treescroll2.set)

        #setrow counter to 0
        varintcounter.set(0)
        self.MyFilebutton6.grid(row=varintcounter.get(),column=0,columnspan=2,sticky="we")
        self.MyStatusLabel.grid(row=varintcounter.get(),column=2,columnspan=7,sticky="nsew")
        self.MyDropDownPack.grid(row=varintcounter.get(),column=8,sticky="nsew")
        self.MyShowPackButton.grid(row=varintcounter.get(),column=9,sticky="nsew")
        self.MyCreatePackButton.grid(row=varintcounter.get(),column=10,sticky="nsew")
        
        #increase the row counter from 0 to 1
        varintcounter.set(varintcounter.get()+1)
        self.MyDataResultFrame1.grid(row=varintcounter.get(),column=0,columnspan=5,rowspan=16,sticky="nsew",padx=5,pady=5)
        self.PyPackRecomResultFrame1.grid(row=varintcounter.get(),column=6,columnspan=5,rowspan=18,sticky="nsew",padx=5,pady=5)

        #increase the row counter from 1 to 2
        varintcounter.set(20)
        self.MyCreateReportbutton1.grid(row=varintcounter.get(),column=9,sticky="we")
        self.MyQuitbutton1.grid(row=varintcounter.get(),column=10,sticky="we")
        

        #Create Column Headers PyPack Result
        PyPackModuleV3.CreateColumnHeadersInChannelSelectionFrame(self.treeview1,self.treescroll1,18,True,False)

        #Create Column Headers for Selection Result
        PyPackModuleV3.CreateColumnHeadersInChannelSelectionFrame(self.treeview2,self.treescroll2,16,True,False)

        self.Run_Button = tk.Button(self.MyDataResultFrame1, 
                                    text = "Run PyPack Recommendation Tool",
                                    bg='Black',fg='White',
                                    command=lambda:PyPackModuleV3.RunPyRecommendationTool(PyPackMasterData,
                                                                                          varstroptmenu.get(),
                                                                                          pd, mathObj,osObj,
                                                                                          varstrfilename.get(),
                                                                                          self.Run_Button,
                                                                                          self.MyCreateReportbutton1,
                                                                                          self.treeview1,
                                                                                          self.treescroll1,
                                                                                          self.treeview2,
                                                                                          self.treescroll2,
                                                                                          self.root,
                                                                                          vartextmessage,varprogressmsg,
                                                                                          varstrfilename,varblnfromfile,
                                                                                          Configurator.MonthlyNetworkFee,
                                                                                          plt,np,
                                                                                          self.MyDataResultFrame1))
        #Disable the Run PyPack tool since the file is not loaded yet
        self.Run_Button.config(height=1, width=70,state=tk.DISABLED)
        #Run_Button.grid(row=int_rec_counter,column=0,columnspan=4, sticky="nsew")
        self.Run_Button.pack()

        varprogressmsg.set("Load your data file to enable the PyPack Recommendation button...")
        #Display the status bar to show the progress of PyPack recommendation tool
        self.Status_Label1 = tk.Label(self.MyDataResultFrame1,textvariable=varprogressmsg, 
                                      bd=1,relief='sunken', anchor='w',bg='Green',fg='White')
        self.Status_Label1.config(height=1, width=70)
        self.Status_Label1.pack()
        #************************************Adding all the controls end code***********************************
        return
    
    def LoadNewWindow(self):
        self.top = tk.Toplevel()
        self.top.title("Creating Your Own Pack")
        self.top.geometry("1250x480")
        #dont allow resizing of the window
        self.top.resizable(False,False)
        self.top.transient(self)
        self.top.config(background="purple")
        self.appc = OpenChildWindow(self.top, self)
        return True
    
    def LoadPackWindow(self):
        self.top = tk.Toplevel()
        self.top.title("Showing Pack Details")
        #self.top.geometry("370x470")
        self.top.geometry("240x470")
        #dont allow resizing of the window
        self.top.resizable(False,False)
        self.top.transient(self)
        self.top.config(background="brown")
        self.appc = OpenPackWindow(self.top, self)
        return 
    
class OpenPackWindow():
    def __init__(self, master, parent):
        self.master = master
        self.frame = tk.Frame(self.master)
        self.parent = parent
        self.widget()
        return
    
    def widget(self):
        #************************************Adding Declare & Initialise Variable begin code************************
        #No local variables
        #************************************Adding Declare & Initialise Variable end code************************
        
        #************************************Adding all the local functions begin code************************
        #declare all local functions
        def getQueryDF(FilterCondition=''):
            dftemp = PyPackMasterData.GetAllSPPackData(pd,self.parent.MyDropDownPack["text"])
            #print('filter condition ='+FilterCondition)
            if FilterCondition != '':    
                #dfReturnData = dftemp[dftemp['Channel_Name'].str.contains('" + FilterCondition + "',na=False)]
                dfReturnData = dftemp[dftemp['Channel_Name'].str.contains('' + FilterCondition + '', regex=True, na=False)]
                #print(dfReturnData.to_string())
                #print(dftemp.to_string())
                return dfReturnData
            else:
                return dftemp
        def FillClearQuery(dftemp, MySearchEntryText, treeview, treescroll):
            #MySearchEntryText.configure(text = "")
            MySearchEntryText.insert(0, '')
            FillQueryPacksData(dftemp, dftemp, treeview, treescroll)
            return
        
        def FillQueryPacksData(dfFiterPack, dftemp, treeview, treescroll):  
   
            #print(dftemp.to_string())
            #Clear the tree first
            treeview.delete(*treeview.get_children())
            treeview.configure(yscrollcommand=treescroll.set)
            #Set the row height and column width
            treeview['height']     = 17
            #treeview['width']     = 17
            treeview['selectmode'] ='extended'

            # Inserting items to the treeview 
            #Display the choices made
            if dfFiterPack.empty :
                treeview.insert('','end',values=("**","There are no packs available for this Service Provider.","**","**"))
            else:

                # Inserting parent
                # Insert first all the Broadcaster details
                dfbroad = dfFiterPack.filter(items=['Broadcaster_y'])
                dfbroad.drop_duplicates(inplace=True)
                int_broad_counter = 0
                treeview.tag_configure("searchtruerow",background='black',foreground='orange')
                treeview.tag_configure("searcfalserow",background='white',foreground='blue')
                
                for broadind in dfbroad.index:
                    #Display selection in Frame 
                    int_broad_counter += 1
                    treeview.insert('',int_broad_counter,'item'+str(int_broad_counter), text=dfbroad['Broadcaster_y'][broadind])
                    #treeview.insert('', '0', 'item1',text ='GeeksforGeeks')

                    #set column width
                    #treeview.column('', width=100, minwidth=55, stretch='no')

                    dfpack = dfFiterPack.query("Broadcaster_y=='" + dfbroad['Broadcaster_y'][broadind] + "'")
                    dfpack = dfpack.filter(items=['Pack_Name2','Pack_Cost_Per_Month'])
                    dfpack.drop_duplicates(inplace=True)
                    int_pack_counter = int_broad_counter*1000
                    for packind in dfpack.index:
                        # Inserting child 
                        int_pack_counter += 1
                        treeview.insert('',int_pack_counter,'item'+str(int_pack_counter), 
                                        text=dfpack['Pack_Name2'][packind] + " ["+ str(dfpack['Pack_Cost_Per_Month'][packind]) +"]")
                        #treeview.insert('', '1', 'item2',text ='Computer Science')

                        # Inserting more than one attribute of an item
                        dfchan = dftemp.query("Pack_Name2=='" + dfpack['Pack_Name2'][packind] + "'")
                        dfchan = dfchan.filter(items=['Channel_Name'])
                        dfchan.drop_duplicates(inplace=True)

                        for chanind in dfchan.index:
                            checkpresence = dfFiterPack['Channel_Name'].isin([dfchan['Channel_Name'][chanind]])
                            if dfFiterPack[checkpresence].empty==False:
                                # Inserting more than one attribute of an item
                                treeview.insert('item'+str(int_pack_counter), 'end',
                                                dfchan['Channel_Name'][chanind]+str(int_pack_counter),
                                                text = dfchan['Channel_Name'][chanind],
                                                tags = ('searchtruerow',))
                            else:
                                # Inserting more than one attribute of an item
                                treeview.insert('item'+str(int_pack_counter), 'end',
                                                dfchan['Channel_Name'][chanind]+str(int_pack_counter),
                                                text = dfchan['Channel_Name'][chanind],
                                                tags = ('searchfalserow',))
                                
                        # Placing each child items in parent widget
                        treeview.move('item'+str(int_pack_counter), 'item'+str(int_broad_counter), 'end') 

            return True
        #************************************Adding all the local functions end code************************
        
        #************************************Adding all the Controls begin code*****************************
        
        self.MySearchLabel    = tk.Label(self.master, text  = "Channel Name",relief='sunken',border=1)
        self.MySearchChannel1 = tk.Entry(self.master, text  = "",relief = 'raised')
        self.MyClosebutton1   = tk.Button(self.master, text = "Close", command=self.master.destroy)
        self.MyClearButton    = tk.Button(self.master, text = "Clear", 
                                          command=lambda:FillClearQuery(getQueryDF(),
                                                                        self.MySearchChannel1,
                                                                        self.tvpack1,
                                                                        self.tspack1))
        self.MySearchButton   = tk.Button(self.master, text = "Search",
        #command=lambda:PyPackModuleV3.FillPacksData(getQueryDF(self.MySearchChannel1.get()),self.tvpack1,self.tspack1)                                              
                                         command=lambda:FillQueryPacksData(getQueryDF(self.MySearchChannel1.get()),
                                                                           getQueryDF(),
                                                                           self.tvpack1,
                                                                           self.tspack1))
        #Create PyPackRecommendations Frame and add treeview1
        self.ShowPackResultFrame1 = tk.LabelFrame(self.master,
                                                  text="Pack Details of "+self.parent.MyDropDownPack["text"],
                                                  padx=0,pady=0)
        self.tvpack1 = ttk.Treeview(self.ShowPackResultFrame1)

        #adding vertical scrollbar
        self.tspack1 = ttk.Scrollbar(self.ShowPackResultFrame1,orient="vertical",command=self.tvpack1.yview)
        self.tspack1.pack(side=RIGHT, fill="y") 
        
        #Configure the treeview to scrollback
        self.tvpack1.pack() 
        self.tvpack1.configure(yscrollcommand=self.tspack1.set)
        
        
        #********************IMPORTANT CODE FOR EVENT BINDING FOR DOUBLE ON CHILD ROWS BEGIN CODE********************
        #Bind the treeview to double click event to fetch the selected row
        #self.tvpack1.bind('<Double 1>',deleteselectedrow)
        #********************IMPORTANT CODE FOR EVENT BINDING FOR DOUBLE ON CHILD ROWS END CODE********************

        #set the row counter 0 to 1
        self.ShowPackResultFrame1.grid(row=0,column=0,columnspan=5,rowspan=18,sticky="nsew",padx=0,pady=0)
        
        #Create Column Headers PyPack Result
        PyPackModuleV3.CreateColumnHeadersInShowPackFrame(self.tvpack1,self.tspack1)

        #increase the row counter from 0 to 1
        self.MySearchLabel.grid(row=18   ,column=0,  sticky="we"  ,padx=5,pady=5)
        self.MySearchChannel1.grid(row=18,column=1,  columnspan=5,  sticky="we",padx=0,pady=0)
        self.MySearchButton.grid(row=19  ,column=0,  columnspan=2,  sticky="we",padx=0,pady=2)
        self.MyClearButton.grid(row=19   ,column=2,  columnspan=3,  sticky="we",padx=0,pady=0)
        self.MyClosebutton1.grid(row=20  ,column=0,  columnspan=5,  sticky="we",padx=1,pady=2)
        
        #************************************Adding all the Controls end code*****************************
        #get all channels data using getQueryDF funtion 
        #load all data for default selection
        PyPackModuleV3.FillPacksData(getQueryDF(), self.tvpack1, self.tspack1)
        
        return True
    
class OpenChildWindow():
    def __init__(self, master, parent):
        self.master = master
        self.frame = tk.Frame(self.master)
        self.parent = parent
        self.widget()
        return
    
    def FillTreeFromTop(self):
        #Adding a local function to add all the selected channels from treeview into a dataframe         
        def GetSelectedChannelsInDF(treeview1):
            #Get all channel selected + mandatory in a treeview_df
            # list of column names here
            df_colnames = ['Channel_Type','Channel_Name','Pack_Cost_Per_Month',
                           'Channel_Genre','Channel_Language','Choose_Yes_No']
            treeview_df = pd.DataFrame(columns=df_colnames)

            #get all channels data
            dftemp = PyPackMasterData.GetAllChannelsData(pd)
            #print(treeview_df)
            for row in treeview1.get_children():
                # each row will come as a list under name "values" 
                values = pd.DataFrame([treeview1.item(row)["values"]], 
                            columns=df_colnames)
                # print(values)
                treeview_df = treeview_df.append(values,sort=True)

            df_mandtemp = dftemp.query("Mandatory=='Yes'")
            df_mandtemp['Choose_Yes_No'] = "Yes"
            df_mandtemp = df_mandtemp.filter(items=df_colnames)
            treeview_df = treeview_df.append(df_mandtemp,sort=True)

            #Filter our channels which are selected as yes
            df_mySelectedChannels     = treeview_df.query("Choose_Yes_No=='Yes'")
            #add susequent 25 chs cost if current selection is found>100
            if df_mySelectedChannels['Channel_Name'].count() >100 :      

                #Add the Fee for subsequent 25 channels
                FeeSub25Chs = (df_mySelectedChannels['Channel_Name'].count()-1)
                #print(FeeSub25Chs)
                FeeSub25Chs = Calculate_NetworkFee_For25SubsequentChannels(mathObj, FeeSub25Chs)
                #print(FeeSub25Chs)
                columnNames = ['Ch_Type','Channel_Name','Pack_Cost_Per_Month','Genre','Language','Choose_Yes_No']
                dftemp = pd.DataFrame([['SD','Fee for Subsequent 25 Chs',FeeSub25Chs,'NA','NA','Yes']],
                                      columns=columnNames)
                df_myCurrentPack = df_mySelectedChannels.append(dftemp,sort=True)

            else:
                #send only the selected channels
                df_myCurrentPack = df_mySelectedChannels

            #print(df_myCurrentPack.to_string())
            return df_myCurrentPack
        
        #df_colnames = ['Channel_Type','Channel_Name','Pack_Cost_Per_Month','Channel_Genre','Channel_Language','Choose_Yes_No']
        #treeview_df = pd.DataFrame(columns=df_colnames)
        df_tp = GetSelectedChannelsInDF(self.treeview1)
        
        #dftemp = treeview_df
        df_tp['Pack_Cost_Per_Month'] = df_tp['Pack_Cost_Per_Month'].apply(pd.to_numeric)
        df_tp = df_tp.sort_values(by=['Pack_Cost_Per_Month'],ascending=[False])
        df_tp = df_tp.reset_index(drop=True)
        
        #Clear the both trees before adding data to it
        self.parent.treeview2.delete(*self.parent.treeview2.get_children())
        self.parent.treeview1.delete(*self.parent.treeview1.get_children())
        
        #Display the choices made
        int_rec_counter = 1
        int_pack_cost = 0
        for ind in df_tp.index:
            #Display selection in Frame
            if (float(df_tp['Pack_Cost_Per_Month'][ind])>0):
                Notes = "Paid-" + df_tp['Channel_Type'][ind] 
            elif(df_tp['Channel_Name'][ind][:2]=='dd'):
                Notes = "Free=Mandatory" 
            else:
                Notes = "Free-" + df_tp['Channel_Type'][ind] 
            if df_tp['Channel_Type'][ind] !="**":
                self.parent.treeview2.insert('','end',text=str(ind), values=(str(ind+1),
                                                                 df_tp['Channel_Name'][ind],
                                                                 str(df_tp['Pack_Cost_Per_Month'][ind]),
                                                                 Notes))
                int_pack_cost += float(df_tp['Pack_Cost_Per_Month'][ind])
                int_rec_counter += 1     
        #Display the pack cost in last row
        self.parent.treeview2.insert('','end',values=("","Total Cost of Paid Channels (Rs)=",round(int_pack_cost,2),"",""))

        #Create Column Headers PyPack Result
        #CreateColumnHeadersInChannelSelectionFrame(self.parent.treeview1,self.parent.treescroll1,18)
        self.parent.MyCreateReportbutton1.configure(state='disabled')
        self.parent.Run_Button.configure(state='normal')
        
        #print("setting blnfromfile flag to No and filename to demoreportfile.txt")
        #set the from file flag as false
        #self.parent.varblnfromfile.set("No")
        #self.parent.varstrfilename.set("demoreportfile.txt")
        self.parent.MyDataResultFrame1.config(text='Create Own Pack Data')
        #self.parent.varprogressmsg.set('Click on Run PyPackRecommTool button to generate the recommendations')
        #self.frame.Status_Label1.config(text='Click on Run PyPackRecommTool button to generate the recommendations')
        #self.parent.Status_Label1.config(text='Click on Run PyPackRecommTool button to generate the recommendations')
        #self.frame.update()
        
        # close child and show the parent windw
        self.master.destroy()
        return
    
    def widget(self):
        #************************************Adding Declare & Initialise Variable begin code************************
        varradopt        = tk.StringVar() # To show only SD or HD channels in tree
        vargenreoptmenu  = tk.StringVar() # To show only selected Genre channels in tree
        varlangoptmenu   = tk.StringVar() # To show only selected language channels in tree
        varmandcheck     = tk.StringVar() # To show all the mandatory channels(dd 23 channels) in tree
        
        #vartextmessage   = tk.StringVar()
        #varprogressmsg   = tk.StringVar()
        
        #Set default values for channel type
        varradopt.set("SD")
        dropGenreOptions = ["Movies","GEC","News","Music","Devotional","Infotainment","Kids",
                            "Miscellaneous","Sports","Lifestyle"]
        #Set default values for genre
        vargenreoptmenu.set(dropGenreOptions[0])
        dropLangOptions = ["English","Hindi","Telugu","Punjabi","Marathi","Bengali",
                           "Gujarati","Tamil","Odia","Malayalam","Bhojpuri",
                           "Kannada","German","Assamese","Rajasthani","French",
                           "Urdu","Haryanvi","Parasi","Nepali","Japanese"]
        #Set default values for language
        varlangoptmenu.set(dropLangOptions[0])
        #Set default values for mandatory as No
        varmandcheck.set('No')
        
        #************************************Adding Declare & Initialise Variable end code************************
        
        #************************************Adding all the local functions begin code************************
        def getrow(event):
            rowid = self.treeview2.identify_row(event.y)
            item  = self.treeview2.item(self.treeview2.focus())
            self.treeview1.tag_configure("costrow",background='orange',foreground='blue')
            self.treeview1.tag_configure("nocostrow",background='blue',foreground='orange')
            self.treeview1.tag_configure("commonrow",background='green',foreground='white')

            if item['values'][0]=="**":
                messagebox.showinfo(title="Invalid Channel",message= "This channel is not valid, cannot add to selection.")
            else:
                #Create list of 'id's
                listOfEntriesInTreeView = self.treeview1.get_children()
                blnDuplicateFound = False
                for each in listOfEntriesInTreeView:          
                    #check if its a duplicate entry compare the added channel 
                    #if its already present
                    if self.treeview1.item(each)['values'][1] == item['values'][1]:
                        messagebox.showerror(title="Duplicate Channel", 
                                             message= item['values'][1] + " channel is already part of your pack selection.")
                        blnDuplicateFound = True
                        break
                if blnDuplicateFound ==False:
                    if float(item['values'][2])>0:
                        self.treeview1.insert('','end',tags=('costrow',),values=(item['values'][0],item['values'][1],item['values'][2],
                                                                         item['values'][3],item['values'][4],'Yes'))
                    else:
                        self.treeview1.insert('','end',tags=('nocostrow',),values=(item['values'][0],item['values'][1],item['values'][2],
                                                                         item['values'][3],item['values'][4],'Yes'))
                #Get the new list of 'id's
                listOfEntriesInTreeView = self.treeview1.get_children() 
                #Display the cost of all selected channels
                int_pack_cost   = 0.0
                int_rec_counter = 1
                if blnDuplicateFound ==False:
                    for each in listOfEntriesInTreeView:
                        #print(treeview.item(each)['values'][col])  #e.g. prints data in clicked cell
                        #treeview.detach(each) #e.g. detaches entry from treeview
                        if self.treeview1.item(each)['values'][0]=="**":
                            self.treeview1.detach(each) #e.g. detaches entry from treeview for total pack cost 
                        else:
                            int_pack_cost   += float(self.treeview1.item(each)['values'][2])
                            int_rec_counter += 1
                    #Display COST at the bottom for the selected channels
                    if int_pack_cost == 0.0:
                        self.treeview1.insert('','end',tags=('commonrow',),
                                         values=("**","Your Pack Has No Paid Channels.","0.0","","",""))
                    else:
                        self.treeview1.insert('','end',tags=('commonrow',),
                                         values=("**","Your Pack Cost =(Rs." + str(round(int_pack_cost,2)) +")",
                                                 str(round(int_pack_cost,2)),"","",""))
            return True

        def deleteselectedrow(event):
            rowid = self.treeview1.identify_row(event.y)
            item  = self.treeview1.item(self.treeview1.focus())
            self.treeview1.tag_configure("costrow",background='orange',foreground='blue')
            self.treeview1.tag_configure("nocostrow",background='blue',foreground='orange')
            self.treeview1.tag_configure("commonrow",background='green',foreground='white')

            #Display the cost of all selected channels
            int_pack_cost   = 0.0
            int_rec_counter = 1
            listOfEntriesInTreeView = self.treeview1.get_children()
            for each in listOfEntriesInTreeView:
                #Delete only if its selected row
                if self.treeview1.item(each)['values'][1]== item['values'][1]:
                    #Delete only if its not a total cost row and not a blank row
                    if self.treeview1.item(each)['values'][0] != "**" and self.treeview1.item(each)['values'][1] != "":
                        self.treeview1.detach(each) #detach entry from treeview for that channel
                        int_rec_counter -= 1
                    elif self.treeview1.item(each)['values'][0] == "**":
                        self.treeview1.detach(each) #detach entry from treeview to remove total cost row
                elif self.treeview1.item(each)['values'][0] == "**":
                    self.treeview1.detach(each) #detach entry from treeview to remove total cost row
                else:
                    int_pack_cost   += float(self.treeview1.item(each)['values'][2])
                    int_rec_counter += 1
                #Display COST at the bottom for the selected channels
            if int_pack_cost == 0.0:
                self.treeview1.insert('','end',tags=('commonrow',),
                                      values=("**","Your Pack Has No Paid Channels.","0.0","","",""))
            else:
                self.treeview1.insert('','end',tags=('commonrow',),
                                      values=("**","Your Pack Cost =(Rs." + str(round(int_pack_cost,2)) +")",
                                              str(round(int_pack_cost,2)),"","",""))
            return True
        
        #************************************Adding all the local functions end code************************
        
        #************************************Adding all the Controls begin code*****************************
        self.MyDropDownGenreMenu1 = tk.OptionMenu(self.master,vargenreoptmenu,*dropGenreOptions,
                                          command=lambda _:PyPackModuleV3.QueryChannelsData(dftemp,
                                                                                          "Channel_Type=='" + varradopt.get().upper()
                                                                                          + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                                                                          + "' and Channel_Genre=='" + vargenreoptmenu.get().lower() 
                                                                                          + "' and Mandatory=='" + varmandcheck.get()
                                                                                          + "'",
                                                                                          self.treeview2,self.treescroll2,
                                                                                          self.treeview1,self.treescroll1,
                                                                                          pd,configtvcol=False))
        
        self.MyDropDownLangMenu1 = tk.OptionMenu(self.master,varlangoptmenu,*dropLangOptions,
                                          command=lambda _:PyPackModuleV3.QueryChannelsData(dftemp,
                                                                                          "Channel_Type=='" + varradopt.get().upper()
                                                                                          + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                                                                          + "' and Channel_Genre=='" + vargenreoptmenu.get().lower() 
                                                                                          + "' and Mandatory=='" + varmandcheck.get()
                                                                                          + "'",
                                                                                          self.treeview2,self.treescroll2,
                                                                                          self.treeview1,self.treescroll1,
                                                                                          pd,configtvcol=False))
        self.CheckButton1 = tk.Checkbutton(self.master, text = "Mandatory",variable = varmandcheck, onvalue='Yes',offvalue='No', 
                                   command=lambda:PyPackModuleV3.QueryChannelsData(dftemp,
                                                                                 "Channel_Type=='" + varradopt.get().upper()
                                                                                 + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                                                                 + "' and Channel_Genre=='" + vargenreoptmenu.get().lower()
                                                                                 + "' and Mandatory=='" + varmandcheck.get()
                                                                                 + "'",
                                                                                 self.treeview2,self.treescroll2,
                                                                                 self.treeview1,self.treescroll1,
                                                                                 pd,configtvcol=False))

        self.RadioButton1 = tk.Radiobutton(self.master, text = "HD",variable = varradopt,value = "HD", 
                                   command=lambda:PyPackModuleV3.QueryChannelsData(dftemp,
                                                                                 "Channel_Type=='" + varradopt.get().upper()
                                                                                 + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                                                                 + "' and Channel_Genre=='" + vargenreoptmenu.get().lower()
                                                                                 + "' and Mandatory=='" + varmandcheck.get()
                                                                                 + "'",
                                                                                 self.treeview2,self.treescroll2,
                                                                                 self.treeview1,self.treescroll1,
                                                                                 pd,configtvcol=False))

        self.RadioButton2 = tk.Radiobutton(self.master, text = "SD",variable = varradopt,value ="SD", 
                                   command=lambda:PyPackModuleV3.QueryChannelsData(dftemp,
                                                                                 "Channel_Type=='" + varradopt.get().upper()
                                                                                 + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                                                                 + "' and Channel_Genre=='" + vargenreoptmenu.get().lower()
                                                                                 + "' and Mandatory=='" + varmandcheck.get()
                                                                                 + "'",
                                                                                 self.treeview2,self.treescroll2,
                                                                                 self.treeview1,self.treescroll1,
                                                                                 pd,configtvcol=False))

        self.MyClosebutton1 = tk.Button(self.master, text = "Close", command=self.master.destroy)
        self.RunPyPackToolbutton1 = tk.Button(self.master, text = "Run PyPack Tool",command=self.FillTreeFromTop)
        
        #Create PyPackRecommendations Frame and add treeview1
        self.PyPackRecomResultFrame1 = tk.LabelFrame(self.master,text="Pack Seletion Details",padx=0,pady=0)
        #treeview1 = ttk.Treeview(PyPackRecomResultFrame1,columns=(1,2,3,4,5),show="headings",height=18,selectmode ='browse')
        self.treeview1 = ttk.Treeview(self.PyPackRecomResultFrame1,columns=("one", "two", "three","four","five"))

        #Create Data Selection Frame to display choice and add treeview2
        self.MyDataResultFrame1      = tk.LabelFrame(self.master,text="Channel Details",padx=5,pady=0)
        #treeview2 = ttk.Treeview(MyDataResultFrame1,columns=(1,2,3,4,5),show="headings",height=16,selectmode ='browse')
        self.treeview2 = ttk.Treeview(self.MyDataResultFrame1,columns=("one", "two", "three","four","five"))

        #adding vertical scrollbar
        self.treescroll1 = ttk.Scrollbar(self.PyPackRecomResultFrame1,orient="vertical",command=self.treeview1.yview)
        self.treescroll1.pack(side=RIGHT, fill="y") 
        self.treescroll2 = ttk.Scrollbar(self.MyDataResultFrame1,orient="vertical",command=self.treeview2.yview)
        self.treescroll2.pack(side=RIGHT, fill="y")

        #Configure the treeview to scrollback
        self.treeview1.pack() 
        self.treeview1.configure(yscrollcommand=self.treescroll1.set)
        self.treeview2.pack()
        self.treeview2.configure(yscrollcommand=self.treescroll2.set)
        
        #********************IMPORTANT CODE FOR EVENT BINDING FOR DOUBLE ON CHILD ROWS BEGIN CODE********************
        #Bind the treeview to double click event to fetch the selected row
        self.treeview2.bind('<Double 1>',getrow)
        self.treeview1.bind('<Double 1>',deleteselectedrow)
        #********************IMPORTANT CODE FOR EVENT BINDING FOR DOUBLE ON CHILD ROWS END CODE********************

        #set the row counter 0 to 1
        self.MyDataResultFrame1.grid(row=0,column=0,columnspan=5,rowspan=16,sticky="nsew",padx=0,pady=5)
        self.PyPackRecomResultFrame1.grid(row=0,column=6,columnspan=5,rowspan=18,sticky="nsew",padx=0,pady=0)

        #Create Column Headers PyPack Result
        PyPackModuleV3.CreateColumnHeadersInChannelSelectionFrame(self.treeview1,self.treescroll1,18,True)

        #Create Column Headers for Selection Result
        PyPackModuleV3.CreateColumnHeadersInChannelSelectionFrame(self.treeview2,self.treescroll2,18,True)

        #increase the row counter from 0 to 1
        self.MyDropDownLangMenu1.grid(row=20,column=0,columnspan=2,sticky="we")
        self.MyDropDownGenreMenu1.grid(row=20,column=2,sticky="we")
        self.RadioButton1.grid(row=20,column=3,sticky="we")
        self.RadioButton2.grid(row=20,column=4,sticky="we")
        self.CheckButton1.grid(row=20,column=5,columnspan=2,sticky="we")
        self.MyClosebutton1.grid(row=20,column=9,columnspan=2,sticky="we")
        self.RunPyPackToolbutton1.grid(row=20,column=7,columnspan=2,sticky="we")
        self.CheckButton1.grid(row=20,column=5,columnspan=2,sticky="we")
        
        #************************************Adding all the Controls end code*****************************
        #get all channels data
        dftemp = PyPackMasterData.GetAllChannelsData(pd)
        
        #load all data for default selection
        PyPackModuleV3.QueryChannelsData(dftemp,"Channel_Type=='" + varradopt.get().upper()
                                       + "' and Channel_Language=='" + varlangoptmenu.get().lower()
                                       + "' and Channel_Genre=='" + vargenreoptmenu.get().lower()
                                       + "' and Mandatory=='" + varmandcheck.get()
                                       + "'",
                                       self.treeview2,self.treescroll2,
                                       self.treeview1,self.treescroll1,
                                       pd,configtvcol=True)
        #***********************************hide the root window BEGIN CODE***************************************************
        #PyPackModuleV3.HideWindow(self.parent,True)
        #***********************************hide the root window END CODE*****************************************************
        return           
    
app = MainApplication(tk.Tk())
app.root.mainloop()


duplicate pack cost savings0
checking pack - Pack no..1...
